<a href="https://colab.research.google.com/github/awaiskhan005/DEEP-LEARNING-AND-FORECASTING/blob/main/KALABAGH_River_Flow_Prediction_usng_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('/content/sample_data/Monthly_Kalabagh_Below-Apr_1971-Dec2020.xlsx')

In [4]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Prefer to use
2,NaN,NaN,NaN,Million ft^3/sec,ft^3/sec,m^3/sec
3,NaN,NaN,Month,Kalabagh Below,Kalabagh Below,Kalabagh Below
4,NaN,NaN,1971-04-01 00:00:00,37.766667,37766.666667,1069.4387


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Load Data (Ensure the file path is correct)
df = pd.read_excel("/content/sample_data/Monthly_Kalabagh_Below-Apr_1971-Dec2020.xlsx", sheet_name='Kalabagh 1971-2020')

# Data Cleaning
df = df.iloc[4:].reset_index(drop=True)
df.columns = ['Index', 'NaN1', 'Date', 'Million ft^3/sec', 'ft^3/sec', 'm^3/sec']
df = df[['Date', 'm^3/sec']].dropna()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.dropna(inplace=True)
scaler = MinMaxScaler()
df['m^3/sec'] = scaler.fit_transform(df[['m^3/sec']])

# Prepare Data
seq_length = 12  # Using past 12 months to predict the next
X, y = [], []
values = df['m^3/sec'].values
for i in range(len(values) - seq_length):
    X.append(values[i:i + seq_length])
    y.append(values[i + seq_length])

X = np.array(X)
y = np.array(y)

# Convert to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Create DataLoader
batch_size = 16
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

# Initialize model, loss, and optimizer
model = LSTMModel(input_size=1, hidden_size=50, num_layers=2, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    for inputs, targets in train_loader:
        inputs = inputs.unsqueeze(-1)  # Reshape to [batch, seq_len, input_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Predict on entire dataset
model.eval()
inputs = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
predictions = model(inputs).detach().numpy()

# Inverse transform predictions
predicted_values = scaler.inverse_transform(predictions)
actual_values = scaler.inverse_transform(y.reshape(-1, 1))

# Convert dates for plotting
plot_dates = df['Date'].values[seq_length:]

# Plot results using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_dates, y=actual_values.flatten(), mode='lines', name='Actual', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=plot_dates, y=predicted_values.flatten(), mode='lines', name='Predicted', line=dict(color='red')))
fig.update_layout(title='Deep Learning Model Prediction for m³/sec',
                  xaxis_title='Date',
                  yaxis_title='m³/sec',
                  legend_title='Legend')
fig.show()


In [7]:
!pip install chronos  # Install the chronos package

  Preparing metadata (setup.py) ... done
  Created wheel for chronos: filename=chronos-0.3-py3-none-any.whl size=7374 sha256=ce66c997f829e3248f6c02c59aa902faad5fe94d9ecff3bce698ef48ee1463aa
  Stored in directory: /root/.cache/pip/wheels/03/cc/04/6c92350bba854bcd3b4692c2bb37c91408537e97dc3fdc049b
Successfully built chronos


In [9]:
pip install chronos-forecasting


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [11]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel  # Using Hugging Face Transformer

# Load Data (Ensure the file path is correct)
df = pd.read_excel("/content/sample_data/Monthly_Kalabagh_Below-Apr_1971-Dec2020.xlsx", sheet_name='Kalabagh 1971-2020')

# Data Cleaning
df = df.iloc[4:].reset_index(drop=True)
df.columns = ['Index', 'NaN1', 'Date', 'Million ft^3/sec', 'ft^3/sec', 'm^3/sec']
df = df[['Date', 'm^3/sec']].dropna()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.dropna(inplace=True)
scaler = MinMaxScaler()
df['m^3/sec'] = scaler.fit_transform(df[['m^3/sec']])

# Prepare Data
seq_length = 12  # Using past 12 months to predict the next
X, y = [], []
values = df['m^3/sec'].values
for i in range(len(values) - seq_length):
    X.append(values[i:i + seq_length])
    y.append(values[i + seq_length])

X = np.array(X)
y = np.array(y)

# Convert to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Create DataLoader
batch_size = 16
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Define Transformer model configuration
config = TimeSeriesTransformerConfig(
    prediction_length=1,
    context_length=seq_length,
    d_model=64,
    nhead=4,
    num_encoder_layers=3,
    num_decoder_layers=3,
)

# Initialize model
model = TimeSeriesTransformerModel(config)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 50
for epoch in range(epochs):
    for inputs, targets in train_loader:
        inputs = inputs.unsqueeze(-1)  # Reshape to [batch, seq_len, input_size]
        past_time_features = torch.zeros(inputs.shape[0], inputs.shape[1], 1)  # Adjusting time feature dimension
        past_observed_mask = torch.ones(inputs.shape[0], inputs.shape[1], 1)  # Adjusting mask dimension
        static_feat = torch.zeros(inputs.shape[0], 1, 1)  # Adding static feature as required

        # Forward pass
        outputs = model(
            past_values=inputs,
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask,
            static_categorical_features=static_feat,  # Adjusting static feature input
            static_real_features=static_feat,
            future_values=None,
            future_time_features=None
        ).last_hidden_state[:, -1, :]
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Predict on entire dataset
model.eval()
inputs = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
past_time_features = torch.zeros(inputs.shape[0], inputs.shape[1], 1)
past_observed_mask = torch.ones(inputs.shape[0], inputs.shape[1], 1)
static_feat = torch.zeros(inputs.shape[0], 1, 1)
predictions = model(
    past_values=inputs,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    static_categorical_features=static_feat,
    static_real_features=static_feat,
    future_values=None,
    future_time_features=None
).last_hidden_state[:, -1, :].detach().numpy()

# Inverse transform predictions
predicted_values = scaler.inverse_transform(predictions)
actual_values = scaler.inverse_transform(y.reshape(-1, 1))

# Convert dates for plotting
plot_dates = df['Date'].values[seq_length:]

# Plot results using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_dates, y=actual_values.flatten(), mode='lines', name='Actual', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=plot_dates, y=predicted_values.flatten(), mode='lines', name='Predicted', line=dict(color='red')))
fig.update_layout(title='Time Series Transformer Prediction for m³/sec',
                  xaxis_title='Date',
                  yaxis_title='m³/sec',
                  legend_title='Legend')
fig.show()


AttributeError: 'TimeSeriesTransformerModel' object has no attribute 'embedder'